In NISEC we have developed a new encryption scheme to help Alice communicate with Bob. The scheme works as follows:

To generate her keys Alice does the following: 

- She chooses a prime $p$, which is used as a modulus;
- She chooses a generator $g$ of a suitable group $\mathbb{G}$;
- She chooses a random $x$ such that $2≤x≤p−2$;
- She outputs her public/private key pair $(pk,sk)$ as $pk=g^x$ and $sk=x$;
- Finally, she publishes the public parameters $(p,g,pk)$, while keeping $sk$ private.

For Bob to send a message to Alice, he does the following:

- He picks a message $m\in\mathbb{N}$;
- He computes $c=pk⋅g^m$;
- He sends $c=(c1,g)$ to Alice.

For Alice to decrypt $c$, she does the following:

- Alice recovers $sk=x$;
- Alice computes $c1⋅g-sk=g^x⋅g^m⋅g−x=g^m$.
  
Moreover, recall that an encryption is called:

- Linear ciphertext homomorphic if:
$$\text{Enc}(pk_i,x_i)⋅ … ⋅\text{Enc}(pk_n,x_n)=\text{Enc}(pk_1 ⋅… ⋅pk_n,x_1+…+x_n)$$

- Linear key homomorphic if:
  
For $n$ public/private key pairs

$$(pk_1, sk_1), … ,(pk_n,sk_n),$$

we can construct a new public/private key pair as:

$$(pk_1⋅ … ⋅pk_n,sk_1+…+sk_n)$$

a) Provide at least two drawbacks of our new encryption scheme;

b) Prove that our scheme is both linear ciphertext and key homomorphic.

For the rest of the exercise, assume that all encrypted messages have small values (i.e. they are not of cryptographic size).

We now assume that $n$
 different users independently encrypt $n$
 different values using our new encryption scheme. That is, the first user ecnrypts $x_1$ using $pk_1$, the second user encrypts $x_2$ using $pk_2$ and so on and so forth. As a next step, all $n$  users send their ciphertexts to a cloud service provider, who already possess the sum $sk_1+…+sk_n$, but not each distinct $sk_i$.

c) Explain how the CSP can compute $x_1+…+x_n$.